<a href="https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/WOS_SCI_SCP_PTJ_CTR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WOS+SCI+SCP+PTJ+CTR

Merge the bibliographic datasets for 
* Web of Science (WOS), 
* Scielo (SCI)
* Scopus  (SCP)
* Puntaje (UDEA)
* Center (CTR)
of the scientific articles of Universidad de Antioquia

For details see [merge.ipynb in Colaboratory](https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/merge.ipynb)

Implementation:
The input pure o partially processed database with WOS-SCI-SCP and may be some UDEA entries from PTJ and Center information with additional data about the Full Name UDEA authors.

Addtionaly UDEA entries can be captured from:
1. A previous WOS-SCI-SCP-UDEA
2. A Data Base with a column with full names (FULL LAST NAMES NAMES, e.g VALDEZ GÚZMAN JUAN ALBERTO) and a list of author Aliases in WOS format (Lastname, Name, e.g Valdez-Gúzman, J.A.) with a list of registered affiliations. TODO: Test
3. The database from Puntaje (UDEA). 

Without PTJ? Check: https://www.kaggle.com/aminer/author-disambiguation

In [1]:
import os
VERSION='NEW'
if os.getcwd()=='/content':
    !pip install openpyxl xlrd wosplus fuzzywuzzy[speedup] > /dev/null

**WARNING!**: `authors_WOS` and `UDEA_authors` will be deleted and rebuided from schatch!

In [2]:
# Delete UDEA_columns and start from schratch
REBUILD=False
MERGE_WITH_TRAINED=True

## functions

In [3]:
import pandas as pd
import swifter
import wosplus as wp
pd.set_option('display.max_colwidth',200)

/usr/local/lib/python3.5/dist-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [4]:
# %load wos_sci_scp_ptj_ctr.py

In [5]:
from wos_sci_scp_ptj_ctr import *

##  Configure public links of  files in Google Drive
* If it is a Google Spreadsheet the corresponding file is downloaded as CSV
* If it is in excel or text file the file is downloaded  directly

To define your  own labeled IDs for public google drive files edit the next cell:

In [6]:
%%writefile drive.cfg
[FILES]
WOS_SCI_SCP_PTJ_CTR.json.gz=19E1C1kRk4I0V3uXojqko8-NEicWaPp1j
WOS_SCP_UDEA_SJR_SIU.xlsx=0BxoOXsn2EUNIQ3R4WDhvSzVLQ2s
Base_de_datos_investigadores_Definitiva.csv=12oalgUeKhpvzkTPBP8pXCeHTrF-KO223dy9ov9w9QKs
UDEA_authors_with_WOS_info.json=1o1eVT4JD0FMMICq_oxrTJOzWh47veBMw
produccion_fecha_vig_2003_2018.xlsx=1WbtX4K__TTLxXRjuLvqUYz9tuHCIlS5v
UDEA_WOS_SCI_SCP_PTJ.json=1OkVytKbxJwGvXZDkynkSoUDtkUOTaT4A

Overwriting drive.cfg


##  Load data bases

In [7]:
affil='Univ Antioquia'
drive_files=wp.wosplus('drive.cfg')

#### DEBUG: if False stop in UDEA_PTJ!!!!

if os.path.exists(UDEAjsonfile):
    UDEA=               pd.read_json(UDEAjsonfile,compression='gzip').reset_index(drop=True)
else:    
    UDEA=drive_files.read_drive_json(UDEAjsonfile,compression='gzip').reset_index(drop=True)

In [8]:
#if REBUILD:
#    !rm WOS_SCI_SCP_PTJ_CTR.json.gz

In [9]:
RECOVER=True #False for test purposes
UDEAjsonfile='WOS_SCI_SCP_PTJ_CTR.json.gz'
#Test purposes
#UDEAjsonfile='UDEA_WOS_SCI_SCP_PTJ.json'
if RECOVER:
    #Requieres latest wosplus!
    tmp=drive_files.load_biblio(UDEAjsonfile,compression='gzip')# TODO CHANGE FOR LAST VERSION IN GOOGLE DRIVE
else:
    tmp=drive_files.load_biblio('UDEAtmp.json')
    #drive_files.load_biblio(
    #  'https://raw.githubusercontent.com/restrepo/medicion/master/cienciometria/data/UDEAtmp300.json'
    #    )#Test: 199+1=200 found
    
UDEA=drive_files.biblio['WOS'].reset_index(drop=True)
#DEBUG
#UDEA=UDEA.sample(300,replace=True).reset_index(drop=True) #Test: 77 found
#tmp=drive_files.load_biblio('Sample_WOS.xlsx')

In [10]:
#TODO: In some future version this columns will be updated only (now they rebuilde)
if 'UDEA_authors' in UDEA:
    UDEA['UDEA_authors']=UDEA['UDEA_authors'].apply(lambda l: l if type(l)==list else None)
else:
    UDEA['UDEA_authors']=None
if 'authors_WOS' in UDEA:
    UDEA['authors_WOS']=UDEA['authors_WOS'].apply(lambda l: l if type(l)==list else [])
else:
    UDEA['authors_WOS']=None

In [11]:
from check_quality import *
check_quality(UDEA)

authors_WOS
13645
Leptonic charged Higgs decays in the Zee model ; authors_WOS: [] ; AU: Sierra, DA
Restrepo, D

Tipo contains UDEA
3239
UDEA_authors
12257
UDEA_authors → full_names (Extrapolado puntaje)
12257
UDEA_authors → "NOMBRE COMPLETO" (Extrapolado CENTRO)
8260


In [12]:
if REBUILD:
    UDEA=clean_institutional_columns(UDEA,prefix='UDEA',Tipo='Tipo')
    UDEA['UDEA_authors']=None

In [13]:
for t in UDEA.Tipo.unique():
    print( '{}:{}'.format( t, UDEA[ UDEA.Tipo==t].shape[0] ) )

WOS_SCP_UDEA:1043
WOS_UDEA:343
SCI_SCP_UDEA:500
WOS_SCI_SCP_UDEA:211
SCP_UDEA:461
WOS_SCI_UDEA:47
SCI_UDEA:634
SCP:2112
WOS:1541
WOS_SCP:4777
WOS_SCI_SCP:557
SCI_SCP:1116
WOS_SCI:100
SCI:2258


In [14]:
UDEA.shape

(15700, 181)

## Load trained old data 

### Merge WOS_SCP_SCI with trained data set PTJ_CTR

Merge requires split in DI and TI. 

if `REBUILD` `False` only empty UDEA_columns are searched for.

In [15]:
SIU=drive_files.read_drive_excel('WOS_SCP_UDEA_SJR_SIU.xlsx')

In [16]:
if MERGE_WITH_TRAINED:
    if os.path.exists('WOS_SCP_UDEA_SJR_SIU.xlsx'):
        SIU=pd.read_excel('WOS_SCP_UDEA_SJR_SIU.xlsx')
    else:    
        SIU=drive_files.read_drive_excel('WOS_SCP_UDEA_SJR_SIU.xlsx')
        
    UDEA,SIU=fill_trained_data(UDEA,SIU)#TODO: Remnove SIU

/usr/local/lib/python3.5/dist-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


15700 (15700, 152)
(7072, 168) (8628, 168)


In [17]:
if MERGE_WITH_TRAINED:
    UDEA.to_json('UDEAtmp.json')
    RECOVER=False
    if RECOVER:
        UDEA=pd.read_json('UDEAtmp.json').reset_index(drop=True)

In [18]:
if 'UDEA_autores' in UDEA.columns and UDEA[UDEA['UDEA_autores']==''].shape[0]:
    UDEA['UDEA_autores']=UDEA['UDEA_autores'].apply(lambda s: pd.np.nan if type(s)==str and s=='' else s)

In [19]:
if 'UDEA_autores' in UDEA.columns:
    print(UDEA[UDEA['UDEA_autores']==''].shape[0],UDEA['UDEA_autores'].dropna().shape[0])

0 7072


In [20]:
check_quality(UDEA)

authors_WOS
13645
Leptonic charged Higgs decays in the Zee model ; authors_WOS: [] ; AU: Sierra, DA
Restrepo, D

Tipo contains UDEA
3239


# Puntaje

Try fill missing UDEA columns with some updated database Puntaje database.

In [21]:
qq=UDEA.copy()

In [22]:
drive_files.biblio['WOS']=qq
drive_files.biblio['WOS'].shape

(15700, 168)

In [23]:
tmp=drive_files.load_biblio('produccion_fecha_vig_2003_2018.xlsx',prefix='UDEA')

In [24]:
pp= drive_files.biblio['UDEA'].copy()

In [25]:
drive_files.biblio['UDEA']=pp

In [26]:
df=merge_puntaje(drive_files)

(32581, 24)
va1 0 0
........................................................................

/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


7258 : 5806 + 2827 = 8633
va2 0 5806
.........................................................

/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


7258 : 5453 + 353 = 5806
va3 0 5453
.......................................................

/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


va4 0 5389
7258 : 5389 + 64 = 5453
(3239, 174) + (5389, 152) = 8628


In [27]:
#TODO: Check why not zero
if 'UDEA_autores' in df.columns:
    print(0,'=',df[df['UDEA_autores']==''].shape[0],'; found:',df['UDEA_autores'].dropna().shape[0])

0 = 0 ; found: 10311


In [28]:
#df['UDEA_autores'].apply(lambda s: pd.np.nan if type(s)==str and s=='' else s).dropna().shape

In [29]:
UDEA=df.copy()

In [30]:
UDEA.shape

(15700, 180)

In [31]:
check_quality(UDEA)

authors_WOS
13645
Leptonic charged Higgs decays in the Zee model ; authors_WOS: [] ; AU: Sierra, DA
Restrepo, D

Tipo contains UDEA
3239


## Creates and fill `'authors_WOS'` json column
A list of dictionaries with author and affiliation from the WOS `C1` column

### Fill C1 for not WOS entries in WOS format and extract  affiliation from C1

In [15]:
#Fill from SCI_C1
UDEA['C1']=SCI_C1_to_C1(UDEA)

In [16]:
#Fill from SCP_C1='SCP_Authors with affiliations
UDEA['C1']=SCP_Authors_with_affiliations_to_C1(UDEA)

In [17]:
UDEA[UDEA['C1'].isnull()].shape

(0, 181)

In [18]:
UDEA[UDEA.Tipo=='WOS'].reset_index(drop=True).C1.loc[0]

'[Dieck, Marianne; Gonzalez Rativa, Maria Claudia] Univ Antioquia, UdeA, Medellin, Antioquia, Colombia.\n'

### Creates `'authors_WOS'` json column

In [19]:
#WARNING: some C1 WOS entries are not normalized: Missing authors
UDEA['authors_WOS']=UDEA.C1.apply(lambda x: x.split('\n') if x else x).apply(
    lambda x:   [y.replace('[','').replace('] ','; ') for y in x if y.find(affil)>-1 ] if x else x ).apply(
     lambda x: get_author_info(x) if x else x)

# Improve normalization: remove C1s with only affiliation (from Scielo)
UDEA['authors_WOS']=UDEA['authors_WOS'].apply( 
    lambda x: [d for d in x if d.get('WOS_author').find(affil)==-1] if type(x)==list else x )

In [20]:
UDEA[UDEA.Tipo=='SCP'].reset_index(drop=True).loc[0].authors_WOS

[{'WOS_author': 'Restrepo, A.',
  'affiliation': ['Mycol. Sect., Dept. Microbiol. Parasitol., Sch. Med., Univ Antioquia, Medellin, Colombia'],
  'i': 0},
 {'WOS_author': 'Arango, M.',
  'affiliation': ['Mycol. Sect., Dept. Microbiol. Parasitol., Sch. Med., Univ Antioquia, Medellin, Colombia'],
  'i': 1},
 {'WOS_author': 'Velez, H.',
  'affiliation': ['Mycol. Sect., Dept. Microbiol. Parasitol., Sch. Med., Univ Antioquia, Medellin, Colombia'],
  'i': 2},
 {'WOS_author': 'Uribe, L.',
  'affiliation': ['Mycol. Sect., Dept. Microbiol. Parasitol., Sch. Med., Univ Antioquia, Medellin, Colombia'],
  'i': 3}]

## Fill `'authors_WOS'` from bad AU, bad C1 and some previous run `UDEA_authors`

In [21]:
kk=UDEA.swifter.apply(lambda row: authors_Wos_from_bad_AU_and_bad_C1(row), axis=1)

In [22]:
UDEA['authors_WOS']=kk

In [23]:
UDEA[UDEA['authors_WOS'].apply(lambda l:len(l))==0].shape

(1099, 181)

## Creates and fill `'UDEA_authors'` json column
List of dictionaries with full authors info, similar to the lens.org author json column
* Includes full institutional data base

### includes `'full_name'` from from UDEA database into `'UDEA_athors'`

In [24]:
#TODO: Remove from fill_trained_data(..)
if 'UDEA_autores' in UDEA.columns:
    UDEA['UDEA_autores']=UDEA['UDEA_autores'].apply(lambda s: s if s else None)    
    UDEA['UDEA_autores']=UDEA['UDEA_autores'].apply(lambda s: re.sub('\s+',' ',s) if type(s)==str else s)
    UDEA['UDEA_authors']=UDEA['UDEA_autores'].apply(lambda s: s.split(';') if type(s)==str else s).apply(
                           lambda l: [{'full_name':y} for y in l ] if type(l)==list else l)

## Load full official researcher data base: PTJ into  `'UDEA_athors'` json column

In [25]:
AU=drive_files.read_drive_excel('Base_de_datos_investigadores_Definitiva.csv')

In [26]:
UPDATE_UDEA_authors_with_AU=True
if (UDEA['UDEA_authors'].dropna().shape[0] and 
    UPDATE_UDEA_authors_with_AU):
    kkn=UDEA.copy()
    kkn=update_institutional_authors(kkn,AU)
    print(kkn.shape,UDEA.shape)
    UDEA=kkn.copy()

0
1
2
3
4
5
6
7
8
9
10
11
(15700, 181) (15700, 181)


Quality check

In [27]:
key_contains_in_list_of_dictionaries(UDEA,'Restrepo, D',column='authors_WOS',key='WOS_author').loc[1:2]

1    [{'i': 0, 'WOS_author': 'Granda-Restrepo, Diana', 'affiliation': ['Univ Antioquia, Fac Quim Farmaceut, Dept Alimentos, Grp Invest Biotecnol Alimentos BIOALI, Medellin, Colombia.']}, {'i': 1, 'WOS_...
2                                                                       [{'WOS_author': 'Restrepo, D.', 'origin': 'from AU+UDEA_authors', 'affiliation': ['Univ Antioquia, Inst Fis, Medellin, Colombia.']}]
Name: authors_WOS, dtype: object

In [28]:
if UPDATE_UDEA_authors_with_AU:
    UDEA.to_json('UDEAtmp.json')
    RECOVER=False
    if RECOVER:
        UDEA=pd.read_json('UDEAtmp.json').reset_index(drop=True)

In [29]:
check_quality(UDEA)

authors_WOS
14601
Leptonic charged Higgs decays in the Zee model ; authors_WOS: [{'WOS_author': 'Restrepo, D.', 'origin': 'from AU+UDEA_authors', 'affiliation': ['Univ Antioquia, Inst Fis, Medellin, Colombia.']}] ; AU: Sierra, DA
Restrepo, D

Tipo contains UDEA
3239
UDEA_authors
10311
UDEA_authors → full_names (Extrapolado puntaje)
10311
UDEA_authors → "NOMBRE COMPLETO" (Extrapolado CENTRO)
6934


## Add `'UDEA.authors_WOS'` info* to `UDEA.UDEA_authors` json column**
(\*) obtained from `UDEA.C1`

(\*\*) Obtained from [puntaje trained old UDEA data](./WOS_SCI_SCP_PTJ_GS_LNS.ipynb#Merge-with-trained-data-set) and the [official researcher list](./WOS_SCI_SCP_PTJ_GS_LNS.ipynb#Merge-with-official-researcher-list)

* Both WOS author alias and affiliation!

### Obtain name parts and initials from `'full_name'` in `UDEA_authors` dictionary and update `UDEA_authors` with them

In [30]:
UDEA['authors_WOS'].dropna().shape,UDEA[UDEA['authors_WOS'].apply(len)!=0]['authors_WOS'].shape

((15700,), (14601,))

In [31]:
UDEA[UDEA['authors_WOS'].apply(len)!=0]['authors_WOS'].shape

(14601,)

In [32]:
import sys
if 'UDEA_authors' not in UDEA.columns and REBUILD==False:
    sys.exit('Make MERGE_WITH_TRAINED True and run again')

### Get list of WOS author aliases from full name.
Example: Juan Alberto Restrepo Camargo: `['Restrepo, J.', 'Restrepo, Juan',...]`

Prepare list

In [33]:
# Obtain spanish name parts from full name
dictupdatetmp=UDEA['UDEA_authors'].apply(lambda x: [y.update( 
                split_full_names(y,full_name='full_name')  ) if not pd.isnull(
                y.get('full_name')) else y for y in x] 
                                   if type(x)==list 
                                   else x)

In [34]:
UDEA['authors_WOS'].dropna().shape,UDEA[UDEA['authors_WOS'].apply(len)==0]['authors_WOS'].shape

((15700,), (1099,))

### Check list aginst real WOS author alias and pick the right one. Capture also affiliation info

In [35]:
kk=UDEA['authors_WOS'].combine( UDEA['UDEA_authors'], func=combinewos )

In [36]:
kk.dropna().shape

(10311,)

In [37]:
UDEA['UDEA_authors']=kk

In [38]:
UDEA['UDEA_authors']=UDEA['UDEA_authors'].apply(lambda l: l if type(l)==list else None)
UDEA_YES=UDEA[~UDEA['UDEA_authors'].isna()].reset_index(drop=True)
UDEA_NOT=UDEA[UDEA['UDEA_authors'].isna()].reset_index(drop=True)
UDEA_NOT['authors_WOS'].dropna().shape

(5389,)

In [39]:
UDEA.to_json('UDEAtmp.json')


### Load output results of previous Cell runs

In [40]:
RECOVER=False
if RECOVER:
    UDEA=pd.read_json('UDEAtmp.json').reset_index(drop=True)

## Build a data base with the unique values of the `'UDEA_authors'` column

Obtain UDEA_authors DataFrame: `aunly`

In [41]:
aunly=DataFrame_authors(UDEA)

ARBOLEDA VASQUEZ LUZ ADRIANA


In [42]:
if not aunly.empty:
    aunly.to_json('UDEA_authors_with_WOS_info.json')

In [43]:
RECOVER=False
if RECOVER:
    UDEA=pd.read_json('UDEAtmp.json').reset_index(drop=True)

In [44]:
UDEA.shape

(15700, 181)

In [45]:
if RECOVER:
    if os.path.exists('UDEA_authors_with_WOS_info.json' ):
        aunly=pd.read_json('UDEA_authors_with_WOS_info.json')
    else:
        aunly=drive_files.read_drive_json('UDEA_authors_with_WOS_info.json')

In [46]:
aunly.shape

(1337, 2)

(1273, 2)

In [47]:
check_quality(UDEA)

authors_WOS
14601
Leptonic charged Higgs decays in the Zee model ; authors_WOS: [{'WOS_author': 'Restrepo, D.', 'origin': 'from AU+UDEA_authors', 'affiliation': ['Univ Antioquia, Inst Fis, Medellin, Colombia.']}] ; AU: Sierra, DA
Restrepo, D

Tipo contains UDEA
3239
UDEA_authors
10311
UDEA_authors → full_names (Extrapolado puntaje)
10311
UDEA_authors → "NOMBRE COMPLETO" (Extrapolado CENTRO)
6934


## Use full list of WOS alias and affiliations  in `'UDEA_authors'` to find to identify WOS alias authors with full insitutional info.
This means that some `None` `UDEA_authors` entries will be filled with the extrapolated information.
* The `'Tipo'` is not updated so the new ones are not part of the PTJ idientified articles!

**NOTE** that there are already `UDEA_authors` not None entries with `'authors_WOS'` entries as empty list: `[]`:

With exact author matching and high `lv.ratio` for affiliation

In [48]:
UDEA['UDEA_authors']=UDEA['UDEA_authors'].apply(lambda l:fill_full_wos_author_info(l,aunly) )

In [49]:
if UDEA['UDEA_authors'].dropna().shape[0]:
    UDEA.to_json('UDEAtmp.json')

In [50]:
RECOVER=False
if RECOVER:
    UDEA=pd.read_json('UDEAtmp.json').reset_index(drop=True)

In [51]:
UDEA.shape

(15700, 181)

In [52]:
kk=UDEA.authors_WOS.combine(UDEA.UDEA_authors,func=lambda x,y: get_UDEA_authors(x,y,aunly))

In [53]:
UDEA.UDEA_authors.dropna().shape

(10311,)

10311...(7072,)

In [54]:
UDEA['UDEA_authors']=kk

In [55]:
UDEA.UDEA_authors.dropna().shape,UDEA.shape

((10940,), (15700, 181))

((10963,), (15704, 181))

In [56]:
aunly.shape

(1337, 2)

(1461, 2)

In [57]:
if not aunly.empty:
    print(aunly.drop_duplicates('tmp_author').shape)

(1337, 2)


In [58]:
if not aunly.empty:
    aunly.to_json('UDEA_authors_with_WOS_info.json')

In [59]:
RECOVER=False
if RECOVER:
    if os.path.exists('UDEA_authors_with_WOS_info.json' ):
        aunly=pd.read_json('UDEA_authors_with_WOS_info.json')
    else:
        aunly=drive_files.read_drive_json('UDEA_authors_with_WOS_info.json')

In [60]:
if UDEA['UDEA_authors'].dropna().shape[0]:
    UDEA.to_json('UDEAtmp.json')

In [61]:
if RECOVER:
    UDEA=pd.read_json('UDEAtmp.json').reset_index(drop=True)

In [62]:
UDEA.to_json('WOS_SCI_SCP_PTJ_CTR.json.gz',compression='gzip')

In [63]:
if 'UDEA_autores' in UDEA.columns:
    print(UDEA[UDEA['UDEA_autores']==''].shape[0],UDEA['UDEA_autores'].dropna().shape[0])

0 10311


In [64]:
if 'UDEA_authors' in UDEA.columns:
    print(UDEA[UDEA['UDEA_authors']==''].shape[0],UDEA['UDEA_authors'].dropna().shape[0])

0 10940


In [65]:
#print 1

In [66]:
check_quality(UDEA)

authors_WOS
14601
Leptonic charged Higgs decays in the Zee model ; authors_WOS: [{'WOS_author': 'Restrepo, D.', 'origin': 'from AU+UDEA_authors', 'affiliation': ['Univ Antioquia, Inst Fis, Medellin, Colombia.']}] ; AU: Sierra, DA
Restrepo, D

Tipo contains UDEA
3239
UDEA_authors
10940
UDEA_authors → full_names (Extrapolado puntaje)
10940
UDEA_authors → "NOMBRE COMPLETO" (Extrapolado CENTRO)
7416


## Exactly as before: Used only if 'UDEA_authors' is empty! 

In [67]:
if RECOVER:
    UDEA=pd.read_json('WOS_SCI_SCP_PTJ_CTR.json.gz',compression='gzip').reset_index(drop=True)
    aunly=drive_files.read_drive_json('UDEA_authors_with_WOS_info.json').reset_index(drop=True)

Same function that `get_UDEA_authors??` but for apply instead of combine

In [68]:
def build_institutional_authors(x,author_df,x_author_key='WOS_author',x_affiliation_key='affiliation',
                                        author_key='WOS_author',
                                        affiliation_key='WOS_affiliation'):
    '''
    Same function that 
    get_UDEA_authors?? 
    but for apply instead of combine
    '''
    if type(x)!=list:
        return None
    ll=[]
    for j in range(len(x)):
        
                                #author_WOS→affiliation always have single affiliation
        kk=find_author_affiliation(x[j].get(x_author_key),x[j].get(x_affiliation_key)[0],
                                        author_df=author_df,
                                        author_key=author_key,
                                        affiliation_key=affiliation_key,
                                        ratio=0.9 )
        if kk:
            ll.append(kk)
    if not ll:
        ll=None
    return ll

In [69]:
if not UDEA['UDEA_authors'].dropna().shape[0]:
    UDEA['UDEA_authors']=UDEA.authors_WOS.apply(lambda l: build_institutional_authors(l,aunly) )

## Find new `'UDEA_authors'` not `None` entries by similarity search in author info:
* names and alias and relaxed similarity in affiliations

### Use similarity in full `'UDEA_authors'` converted as string

In [70]:
# Normalize
UDEA['authors_WOS']=UDEA['authors_WOS'].apply(lambda l: [] if not l else l)
UDEA['UDEA_authors']=UDEA['UDEA_authors'].apply(lambda l: l if type(l)==list else None)

In [71]:
#Test
UDEA_YES=UDEA[~UDEA['UDEA_authors'].isna()].reset_index(drop=True)
UDEA_NOT=UDEA[UDEA['UDEA_authors'].isna()].reset_index(drop=True)
UDEA_NOT['authors_WOS'].dropna().shape

(4760,)

In [72]:
UDEA[~UDEA['UDEA_authors'].isna()].shape,UDEA['UDEA_authors'].dropna().shape

((10940, 181), (10940,))

In [73]:
df2=aunly.copy()
df2=pd.DataFrame( list( df2['UDEA_authors'].values ) )
df2['UDEA_authors']=aunly['UDEA_authors']
contents=df2[['WOS_author','WOS_affiliation','UDEA_authors','full_name']].reset_index(drop=True)
contents['WOS_author']=contents['WOS_author'].astype(str)
contents['WOS_affiliation']=contents['WOS_affiliation'].astype(str)

In [74]:
UDEA_NOT['authors_WOS'].dropna().shape

(4760,)

dfnot=UDEA_NOT.copy()
dfnot=dfnot.reset_index(drop=True)

In [75]:
import pandas as pd
import swifter

UDEA_YES.shape

####  Find institutional author info with safe json column converted into string

dfnot['UDEA_authors'].shape#.apply(lambda l: l if type(l)==list else None)

In [76]:
%time kk=UDEA.swifter.apply(lambda r: json_fuzzy_merge_full(r,contents,cutoff_extra=65), axis=1)


CPU times: user 7min 23s, sys: 1.62 s, total: 7min 25s
Wall time: 7min 24s


In [77]:
#%time kk=dfnot['authors_WOS'].apply(lambda l: json_fuzzy_merge_full(l,contents,cutoff_extra=65) if type(l)==list else l)

In [78]:
kk.dropna().shape

(11657,)

In [79]:
UDEA['UDEA_authors']=kk

In [80]:
UDEA_YES=UDEA[~UDEA['UDEA_authors'].isna()].reset_index(drop=True)
UDEA_NOT=UDEA[UDEA['UDEA_authors'].isna()].reset_index(drop=True)

In [81]:
UDEA[~UDEA['UDEA_authors'].isna()].shape,UDEA['UDEA_authors'].dropna().shape

((11657, 181), (11657,))

In [82]:
UDEA_NOT.shape[0]+UDEA_YES.shape[0],UDEA_NOT.shape[0],'+',UDEA_YES.shape[0]

(15700, 4043, '+', 11657)

###  Find new `'UDEA_authors entries'`  entries with similarity in  json column specific keys
* Similarity in full names and WOS alias with relaxed similarity in affiliation  but extra information: Names of author  journals used (WOS: `SO` column)

In [83]:
df2=aunly.copy()
df2=pd.DataFrame( list( df2['UDEA_authors'].values ) )
df2['UDEA_authors']=aunly['UDEA_authors']
contents=df2[['WOS_author','WOS_affiliation','UDEA_authors']].reset_index(drop=True)
contents['WOS_author']=contents['WOS_author']#.astype(str)
contents['WOS_affiliation']=contents['WOS_affiliation']#.astype(str)

import pandas as pd
import swifter
import wosplus as wp
pd.set_option('display.max_colwidth',200)
import numpy as np
import fuzzywuzzy.process as fwp
from fuzzywuzzy import fuzz


#json_column='UDEA_authors'
#UDEA_NOT=UDEA[UDEA[json_column]==''].reset_index(drop=True)
#UDEA_YES=UDEA[UDEA[json_column]!=''].reset_index(drop=True)
#print( UDEA_NOT.shape[0],UDEA_YES.shape[0] )

df2=aunly.copy()
df2=pd.DataFrame( list( df2['UDEA_authors'].values ) )
df2['UDEA_authors']=aunly['UDEA_authors']
contents=df2[['WOS_author','WOS_affiliation','UDEA_authors']].reset_index(drop=True)
contents['WOS_author']=contents['WOS_author']#.astype(str)
contents['WOS_affiliation']=contents['WOS_affiliation']#.astype(str)



In [84]:
UDEA['authors_WOS']=UDEA['authors_WOS'].apply(lambda l: [] if not l else l)
UDEA['UDEA_authors']=UDEA['UDEA_authors'].apply(lambda l: l if type(l)==list else None)

In [ ]:
kkk=UDEA.swifter.apply(lambda row: json_fuzzy_merge(row,UDEA,contents),axis=1)

In [ ]:
UDEA['UDEA_authors']=kkk

In [ ]:
UDEA_YES=UDEA[~UDEA['UDEA_authors'].isna()].reset_index(drop=True)
UDEA_NOT=UDEA[UDEA['UDEA_authors'].isna()].reset_index(drop=True)

In [96]:
UDEA_NOT.shape

(3376, 181)

In [ ]:
UDEA.shape

In [ ]:
UDEA_YES=UDEA[~UDEA['UDEA_authors'].isna()].reset_index(drop=True)
UDEA_NOT=UDEA[UDEA['UDEA_authors'].isna()].reset_index(drop=True)

In [ ]:
UDEA[~UDEA['UDEA_authors'].isna()].shape,UDEA['UDEA_authors'].dropna().shape

In [97]:
UDEA['UDEA_authors'].dropna().shape

(12324,)

(12257,)

Quality checks

UDEA_YES=UDEA[UDEA.UDEA_nombre!=''].reset_index(drop=True)
UDEA_NOT=UDEA[UDEA.UDEA_nombre==''].reset_index(drop=True)

UDEA_YES['Tipo']=UDEA_YES.Tipo.str.replace('([SW][CO][SIP])$',r'\1_UDEA')

UDEA=UDEA_YES.append(UDEA_NOT)
UDEA=UDEA.reset_index(drop=True)

UDEA[UDEA.Tipo.str.contains('UDEA')].shape

In [ ]:
UDEA.to_json('WOS_SCI_SCP_PTJ_CTR.json.gz',compression='gzip')

In [ ]:
check_quality(UDEA)

In [ ]:
print 1

## Try other approachs

In [ ]:
wp.merge_with_close_matches??

In [ ]:
%%writefile test.cfg
[FILES]
Sample_WOS.xlsx = 1--LJZ4mYyQcaJ93xBdbnYj-ZzdjO2Wq2
Sample_SCI.xlsx = 1-3a-hguQTk5ko8JRLCx--EKaslxGVscf
Sample_SCP.xlsx = 1-IAWlMdp2U-9L2jvZUio04ub1Ym3PX-H

In [ ]:
cib=wp.wosplus('test.cfg')
#cib.Debug=True
cib.load_biblio('Sample_WOS.xlsx')
cib.load_biblio('Sample_SCI.xlsx',prefix='SCI')
cib.load_biblio('Sample_SCP.xlsx',prefix='SCP')

In [ ]:
def get_close_matches_Levenshtein(
        word,
        possibilities,
        n=3,
        cutoff=0.6,
        full=False):
    '''Replaces difflib.get_close_matches with faster algortihm based on
       Levenshtein.ratio.
       HINT: Similarity increase significatively after lower() and unidecode()

       Refs: https://en.wikipedia.org/wiki/Levenshtein_distance
    '''
    import pandas as pd
    import Levenshtein
    if isinstance(possibilities, str):
        possibilities = [possibilities]
    rs = pd.DataFrame()
    MATCH = False
    for p in possibilities:
        similarity = Levenshtein.ratio(word, p)
        # print(word,'::',p,similarity)
        # sys.exit()
        if similarity >= cutoff:
            MATCH = True
            rs = rs.append({'similarity': similarity,
                            'match': p}, ignore_index=True)

    if MATCH:
        rs = rs.sort_values(
            'similarity', ascending=False).reset_index(drop=True)
        if full:
            return list(rs['match'][:n].values), list(
                rs['similarity'][:n].values)
        else:
            return list(rs['match'][:n].values)
    else:
        if full:
            return ([], 0)
        else:
            return []

In [ ]:
scorer=fuzz.ratio

In [ ]:
def get_close_matches_Levenshtein_new(
        word,
        possibilities,
        n=1,
        cutoff=0.6,
        full=False,
        scorer=fuzz.ratio): #
    r=fwp.extract(word,possibilities,scorer=scorer,limit=n)
    
    if r[0][1]/100.>cutoff:
        if full:
            return [t[0] for t in r],[t[1]/100. for t in r]
        else:
            return [t[0] for t in r]
    else:
        if full:
            return ([], 0)
        else:
            return []        

In [ ]:
def merge_with_close_matches_old(
        left,
        right,
        left_on='ST',
        right_on='UDEA_simple_título',
        left_extra_on='SO',
        right_extra_on='UDEA_nombre revista o premio',
        how='inner',
        n=1,
        cutoff=0.6,
        full=True,
        cutoff_extra=0.6):
    '''For each entry of the column: left_on of DataFrame left (cannot have empty fields),
       try to find the close match inside each row of right DataFrame, by comparing with
       the right_on entry of the row. When a row match is found, the full right row is appended
       to the matched row in the left DataFrame.
       If the similarity between the entries at left_on and right_on is less than 0.8,
       an additional check is performed between the entries left_extra_on and right_extra_on
       of the matched row.

       how implemented: inner and left (Default: inner)
    '''
    import numpy as np
    from unidecode import unidecode
    import pandas as pd
    # import sys #import globally
    # print(left[left_on][0])
    # sys.exit()
    words = left[left_on].str.lower().map(unidecode)
    possibilities = right[right_on].str.lower().map(unidecode)

    joined = pd.DataFrame()
    mi = np.array([])
    for i in left.index:
        if i % 100 == 0:
            print('.', end="")
        joined_series = left.loc[i]
        #joined_series=joined_series.append(pd.Series( {similarity_column:0} ))
        title, similarity = get_close_matches_Levenshtein(
            words[i], possibilities, n=n, cutoff=cutoff, full=full)
        # print(i,words[i],title,similarity) #cutuff 0.6 0.7 0.8 0.85 0.91 0.95
        # sys.exit()
        if title:
            mtch = right[possibilities == title[0]]
            # >=cutoff, e.g 0.65 0.95 0.81 0.86 0.9 0.96
            chk_cutoff = similarity[0]
            crosscheck = cutoff + 0.2  # 0.8 # e.g. 0.8 0.9 0.9 0.9 0.9 0.9
            if crosscheck >= 1:
                # force check if match worst than this (by experience)
                crosscheck = 0.95
            if chk_cutoff < crosscheck:  # e.g 0.65<0.8 0.95~<0.9 0.81~<0.0 0.86<0.9 0.91<~0.9 0.96~<0.9
                if get_close_matches_Levenshtein(unidecode(left[left_extra_on][i].lower()), [unidecode(
                        mtch[right_extra_on][mtch.index[0]].lower())], cutoff=cutoff_extra):  # cutoff=0.6
                    chk_cutoff = crosscheck + 0.1

            if chk_cutoff >= crosscheck:
                joined_series = joined_series.append(mtch.loc[mtch.index[0]])
                if how == 'outer':
                    mi = np.concatenate((mi, mtch.index.values))
                # joined_series[similarity_column]=similarity[0]

            #return joined_series
            if how == 'inner':
                joined = joined.append(joined_series, ignore_index=True)

        if (how == 'left' or 'outer'):
            joined = joined.append(joined_series, ignore_index=True)
    if how == 'outer':
        joined = joined.append(right.drop(
            right.index[list(mi.astype(int))]).reset_index(drop=True))
    return joined

def merge_with_close_matches_new(
        left,
        right,
        left_on='ST',
        right_on='UDEA_simple_título',
        left_extra_on='SO',
        right_extra_on='UDEA_nombre revista o premio',
        how='inner',
        n=1,
        cutoff=0.6,
        full=True,
        cutoff_extra=0.7):
    '''For each entry of the column: left_on of DataFrame left (cannot have empty fields),
       try to find the close match inside each row of right DataFrame, by comparing with
       the right_on entry of the row. When a row match is found, the full right row is appended
       to the matched row in the left DataFrame.
       If the similarity between the entries at left_on and right_on is less than 0.8,
       an additional check is performed between the entries left_extra_on and right_extra_on
       of the matched row.

       how implemented: inner and left (Default: inner)
    '''
    import numpy as np
    from unidecode import unidecode
    import pandas as pd
    # import sys #import globally
    # print(left[left_on][0])
    # sys.exit()
    words = left[left_on].str.lower().map(unidecode)
    possibilities = right[right_on].str.lower().map(unidecode)

    joined = pd.DataFrame()
    mi = np.array([])
    for i in left.index:
        if i % 100 == 0:
            print('.', end="")
        joined_series = left.loc[i]
        #joined_series=joined_series.append(pd.Series( {similarity_column:0} ))
        title, similarity = get_close_matches_Levenshtein_new(
            words[i], possibilities, n=n, cutoff=cutoff, full=full)
        # print(i,words[i],title,similarity) #cutuff 0.6 0.7 0.8 0.85 0.91 0.95
        # sys.exit()
        if title:
            mtch = right[possibilities == title[0]]
            # >=cutoff, e.g 0.65 0.95 0.81 0.86 0.9 0.96
            chk_cutoff = similarity[0]
            crosscheck = cutoff + 0.2  # 0.8 # e.g. 0.8 0.9 0.9 0.9 0.9 0.9
            if crosscheck >= 1:
                # force check if match worst than this (by experience)
                crosscheck = 0.95
            if chk_cutoff < crosscheck:  # e.g 0.65<0.8 0.95~<0.9 0.81~<0.0 0.86<0.9 0.91<~0.9 0.96~<0.9
                if get_close_matches_Levenshtein_new(unidecode(left[left_extra_on][i].lower()), [unidecode(
                        mtch[right_extra_on][mtch.index[0]].lower())], cutoff=cutoff_extra):  # cutoff=0.6
                    chk_cutoff = crosscheck + 0.1

            if chk_cutoff >= crosscheck:
                joined_series = joined_series.append(mtch.loc[mtch.index[0]])
                if how == 'outer':
                    mi = np.concatenate((mi, mtch.index.values))
                # joined_series[similarity_column]=similarity[0]

            #return joined_series
            if how == 'inner':
                joined = joined.append(joined_series, ignore_index=True)

        if (how == 'left' or 'outer'):
            joined = joined.append(joined_series, ignore_index=True)
    if how == 'outer':
        joined = joined.append(right.drop(
            right.index[list(mi.astype(int))]).reset_index(drop=True))
    return joined

In [ ]:
cib.biblio['WOS']=UDEA.sample(500).reset_index(drop=True).copy().fillna('')
cib.biblio['SCI']=SIU[0:100].copy().fillna('')

In [ ]:
tmp=drive_files.load_biblio('produccion_fecha_vig_2003_2018.xlsx',prefix='UDEA')

In [ ]:
drive_files.biblio['UDEA']

In [ ]:
%time kkold=merge_with_close_matches_old(cib.biblio['WOS'],cib.biblio['SCI'].drop('Tipo',axis='columns'),left_on='TI',right_on='SCI_TI',right_extra_on='SCI_SO',how='left')


In [ ]:
%time kknew=merge_with_close_matches_new(cib.biblio['WOS'],cib.biblio['SCI'].drop('Tipo',axis='columns'),left_on='TI',right_on='SCI_TI',right_extra_on='SCI_SO',how='left')


In [ ]:
kkold.shape,kknew.shape

In [ ]:
(kkold['SCI_TI'].apply(lambda s: s if s else pd.np.nan).dropna().shape,
 kknew['SCI_TI'].apply(lambda s: s if s else pd.np.nan).dropna().shape)

In [ ]:
kknew[['TI','SCI_TI']].dropna()

In [ ]:
kkold[['TI','SCI_TI']].dropna()

In [ ]:
import time 

In [ ]:
s=time.time()
kkold=merge_with_close_matches_old(cib.biblio['WOS'][['TI','SO']],drive_files.biblio['UDEA'][['UDEA_título',
                                                            'UDEA_nombre revista o premio']],
                            left_on='TI',left_extra_on='SO',right_on='UDEA_título',
                            right_extra_on='UDEA_nombre revista o premio',how='left')
print(time.time()-s)

In [ ]:
s=time.time()
kknew=merge_with_close_matches_new(cib.biblio['WOS'][['TI','SO']],drive_files.biblio['UDEA'][['UDEA_título',
                                                            'UDEA_nombre revista o premio']],
                            left_on='TI',left_extra_on='SO',right_on='UDEA_título',
                            right_extra_on='UDEA_nombre revista o premio',how='left')
print(time.time()-s)

In [ ]:
..................

In [ ]:
(kkold['UDEA_título'].apply(lambda s: s if s else pd.np.nan).dropna().shape,
 kknew['UDEA_título'].apply(lambda s: s if s else pd.np.nan).dropna().shape)